Линейная регрессия
===========

In [217]:
import pandas as pd
#import pandas_profiling
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [218]:
### Зависит ли обилие птиц в лесах Австралии от характеристик леса? 
### (Loyn,1987, пример из кн. Quinn, Keough, 2002) 56 лесных участков в юго-восточной Виктории, Австралия 
### AREA - Площадь леса, га (логарифм)
### DIST - Расстояние до ближайшего леса, км (логарифм)
### LDIST - Расстояние до ближайшего леса большего размера, км (логарифм)
### YR.ISOL - Год начала изоляции
### GRAZE -  история выпаса скота от 1 (легкий) до 5 (тяжелый)
### ALT - средняя высота (м)
### ABUND - Обилие птиц


In [219]:
df = pd.read_csv('loyn1.csv', sep=';', index_col=0, decimal=',')


In [220]:
df.head()

,ABUND,AREA,DIST,LDIST,YR.ISOL,GRAZE,ALT
Site,,,,,,,
1,5.3,0.1,39,39,1968,2,160
2,2.0,0.5,234,234,1920,5,60
3,1.5,0.5,104,311,1900,5,140
4,17.1,1.0,66,66,1966,3,160
5,13.8,1.0,246,246,1918,5,140


In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 1 to 56
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ABUND    56 non-null     float64
 1   AREA     56 non-null     float64
 2   DIST     56 non-null     int64  
 3   LDIST    56 non-null     int64  
 4   YR.ISOL  56 non-null     int64  
 5   GRAZE    56 non-null     int64  
 6   ALT      56 non-null     int64  
dtypes: float64(2), int64(5)
memory usage: 3.5 KB


Прежде, чем строить модель, нужно решить вопрос с пропущенными значениями.

In [222]:
df.isnull().sum()

ABUND      0
AREA       0
DIST       0
LDIST      0
YR.ISOL    0
GRAZE      0
ALT        0
dtype: int64

In [223]:
#pd.read_csv('loyn1.csv', sep=';', index_col=0, decimal=',').profile_report()

#### Построение модели

Теперь можно переходить непосредственно к построению модели.

In [224]:
from sklearn.linear_model import LinearRegression

In [225]:
df['AREA'] = np.log(df['AREA'])
df['DIST'] = np.log(df['DIST'])
df['LDIST'] = np.log(df['LDIST'])

X = df.drop('ABUND', axis=1)
y = df['ABUND']


In [226]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

Считаем качество модели (коэффициент $R^2$).

In [227]:
print(model.score(X, y))

0.6849359484494817


Коэффициент детерминации увеличился, значит качество модели улучшилось. Обилие птиц больше зависит от логарифмических показателей площади, дистанции до ближайшего леса и дистанции до ближайщего леса большей площади.

Выведем регрессионные коэффициенты с помощью метода ```coef_``` и свободный член с помощью метода ```intercept_```.

In [228]:
coef = pd.DataFrame(zip(['intercept'] + X.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
coef

,predictor,coef
0,intercept,-125.697246
1,AREA,3.244278
2,DIST,-0.393886
3,LDIST,-0.281605
4,YR.ISOL,0.073871
5,GRAZE,-1.667738
6,ALT,0.019508


В ```sklearn``` не предусмотрена процедура определения статистической значимости регрессионных коэффициентов. Поэтому воспользуемся моделью из пакета ```statsmodels```.

In [229]:
from scipy import stats

def regression_coef(model, X, y):
    coef = pd.DataFrame(zip(['intercept'] + X.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
    X1 = np.append(np.ones((len(X),1)), X, axis=1)
    b = np.append(model.intercept_, model.coef_)
    MSE = np.sum((model.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    var_b = MSE * (np.linalg.inv(np.dot(X1.T, X1)).diagonal())
    sd_b = np.sqrt(var_b)
    t = b / sd_b
    coef['pvalue'] = [2 * (1 - stats.t.cdf(np.abs(i), (len(X1) - 1))) for i in t]
    return coef
regression_coef(model, X, y)

,predictor,coef,pvalue
0,intercept,-125.697246,0.171713
1,AREA,3.244278,0.000004
2,DIST,-0.393886,0.733356
3,LDIST,-0.281605,0.758814
4,YR.ISOL,0.073871,0.104489
5,GRAZE,-1.667738,0.075507
6,ALT,0.019508,0.414345


Минимаксная нормализация для логарифмических значений (возникают проблемы при подсчете коэффициентов)

In [230]:
maxAREA = df['AREA'].max()
minAREA = df['AREA'].min()
kfAREA = maxAREA - minAREA

maxDIST = df['DIST'].max()
minDIST = df['DIST'].min()
kfDIST = maxDIST - minDIST

maxLDIST = df['LDIST'].max()
minLDIST = df['LDIST'].min()
kfLDIST = maxLDIST - minLDIST

print(kfAREA, kfDIST, kfLDIST)

9.781884730776879 4.005233079455355 5.137154982589513


In [231]:
normAREA = [0]
for el in df['AREA']:
        el = (el - minAREA)/kfAREA
        normAREA.append(el)
df['normAREA'] = pd.DataFrame(data = normAREA)

normDIST = [0]
for el in df['DIST']:
        el = (el - minDIST)/kfDIST
        normDIST.append(el)
df['normDIST'] = pd.DataFrame(data = normDIST)

normLDIST = [0]
for el in df['LDIST']:
        el = (el - minLDIST)/kfLDIST
        normLDIST.append(el)
df['normLDIST'] = pd.DataFrame(data = normLDIST)
print(df)

      ABUND      AREA      DIST     LDIST  YR.ISOL  GRAZE  ALT  normAREA  \
Site                                                                       
1       5.3 -2.302585  3.663562  3.663562     1968      2  160  0.000000   
2       2.0 -0.693147  5.455321  5.455321     1920      5   60  0.164532   
3       1.5 -0.693147  4.644391  5.739793     1900      5  140  0.164532   
4      17.1  0.000000  4.189655  4.189655     1966      3  160  0.235393   
5      13.8  0.000000  5.505332  5.505332     1918      5  140  0.235393   
6      14.1  0.000000  5.455321  5.652489     1965      3  130  0.235393   
7       3.8  0.000000  6.146329  6.146329     1955      5   90  0.235393   
8       2.2  0.000000  5.648974  7.511525     1920      5   60  0.235393   
9       3.3  0.000000  5.049856  5.049856     1965      4  130  0.235393   
10      3.0  0.000000  5.739793  6.347389     1900      5  130  0.235393   
11     27.6  0.693147  4.189655  5.805135     1926      3  210  0.306253   
12      1.8 

In [232]:
Xn = df.drop('ABUND', axis=1)
yn = df['ABUND']

In [233]:
model = LinearRegression()
model.fit(Xn, yn)

LinearRegression()

In [234]:
print(model.score(Xn, yn))

0.6849359484494819


In [235]:
coef = pd.DataFrame(zip(['intercept'] + Xn.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
coef

,predictor,coef
0,intercept,-125.883310
1,AREA,3.210723
2,DIST,-0.370773
3,LDIST,-0.271324
4,YR.ISOL,0.073871
5,GRAZE,-1.667738
6,ALT,0.019508
7,normAREA,0.328232
8,normDIST,-0.092572
9,normLDIST,-0.052816


In [236]:
regression_coef(model, Xn, yn)

<ipython-input-229-827d510938b9>:10: RuntimeWarning: invalid value encountered in sqrt
  sd_b = np.sqrt(var_b)
C:\Python\lib\site-packages\scipy\stats\_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,predictor,coef,pvalue
0,intercept,-125.883310,NaN
1,AREA,3.210723,NaN
2,DIST,-0.370773,NaN
3,LDIST,-0.271324,NaN
4,YR.ISOL,0.073871,0.115225
5,GRAZE,-1.667738,0.084582
6,ALT,0.019508,0.428624
7,normAREA,0.328232,NaN
8,normDIST,-0.092572,NaN
9,normLDIST,-0.052816,NaN


Минимаксная нормализация для нелогарифмированных значений (однако она нерациональная, потому что потом нельзя прологарифмировать)

In [237]:
dfnorm = pd.read_csv('loyn1.csv', sep=';', index_col=0, decimal=',')

In [238]:
dfnorm.head()

,ABUND,AREA,DIST,LDIST,YR.ISOL,GRAZE,ALT
Site,,,,,,,
1,5.3,0.1,39,39,1968,2,160
2,2.0,0.5,234,234,1920,5,60
3,1.5,0.5,104,311,1900,5,140
4,17.1,1.0,66,66,1966,3,160
5,13.8,1.0,246,246,1918,5,140


In [239]:
maxAREA = dfnorm['AREA'].max()
minAREA = dfnorm['AREA'].min()
kfAREA = maxAREA - minAREA

maxDIST = dfnorm['DIST'].max()
minDIST = dfnorm['DIST'].min()
kfDIST = maxDIST - minDIST

maxLDIST = dfnorm['LDIST'].max()
minLDIST = dfnorm['LDIST'].min()
kfLDIST = maxLDIST - minLDIST

print(kfAREA, kfDIST, kfLDIST)
print(dfnorm['AREA'])

1770.9 1401 4400
Site
1        0.1
2        0.5
3        0.5
4        1.0
5        1.0
6        1.0
7        1.0
8        1.0
9        1.0
10       1.0
11       2.0
12       2.0
13       2.0
14       2.0
15       2.0
16       2.0
17       3.0
18       3.0
19       4.0
20       4.0
21       4.0
22       4.0
23       5.0
24       5.0
25       6.0
26       6.0
27       7.0
28       7.0
29       8.0
30       9.0
31      10.0
32      11.0
33      12.0
34      12.0
35      13.0
36      15.0
37      17.0
38      18.0
39      19.0
40      22.0
41      26.0
42      29.0
43      32.0
44      34.0
45      40.0
46      44.0
47      48.0
48      49.0
49      50.0
50      57.0
51      96.0
52     108.0
53     134.0
54     144.0
55     973.0
56    1771.0
Name: AREA, dtype: float64


In [240]:
normAREA = [0]
for el in dfnorm['AREA']:
        el = (el - minAREA)/kfAREA
        normAREA.append(el)
dfnorm['normAREA'] = pd.DataFrame(data = normAREA)

In [241]:
normDIST = [0]
for el in dfnorm['DIST']:
        el = (el - minDIST)/kfDIST
        normDIST.append(el)
dfnorm['normDIST'] = pd.DataFrame(data = normDIST)


In [242]:
normLDIST = [0]
for el in dfnorm['LDIST']:
        el = (el - minLDIST)/kfLDIST
        normLDIST.append(el)
dfnorm['normLDIST'] = pd.DataFrame(data = normLDIST)
print(dfnorm)

      ABUND    AREA  DIST  LDIST  YR.ISOL  GRAZE  ALT  normAREA  normDIST  \
Site                                                                        
1       5.3     0.1    39     39     1968      2  160  0.000000  0.009279   
2       2.0     0.5   234    234     1920      5   60  0.000226  0.148465   
3       1.5     0.5   104    311     1900      5  140  0.000226  0.055675   
4      17.1     1.0    66     66     1966      3  160  0.000508  0.028551   
5      13.8     1.0   246    246     1918      5  140  0.000508  0.157031   
6      14.1     1.0   234    285     1965      3  130  0.000508  0.148465   
7       3.8     1.0   467    467     1955      5   90  0.000508  0.314775   
8       2.2     1.0   284   1829     1920      5   60  0.000508  0.184154   
9       3.3     1.0   156    156     1965      4  130  0.000508  0.092791   
10      3.0     1.0   311    571     1900      5  130  0.000508  0.203426   
11     27.6     2.0    66    332     1926      3  210  0.001073  0.028551   